In [1]:
#Import packages

#Rdkit
from rdkit import Chem

#Pandas
import pandas as pd
from pandas import DataFrame

#Others
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

import joblib
import random
import numpy as np
import os
import math
import warnings

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from plotnine import *

In [2]:
def print_RxC(dataframe):
    rows= dataframe.shape[0]
    columns= dataframe.shape[1]
    rows_columns=print(dataframe.shape[0],"Moleculars ×",dataframe.shape[1],"Descriptors")
    return rows_columns
def trimm_correlated(df_in, threshold):
    df_corr = df_in.corr(method='spearman')
    df_not_correlated = ~(df_corr.mask(np.tril(np.ones([len(df_corr)]*2, dtype=bool))).abs() > threshold).any()
    un_corr_idx = df_not_correlated.loc[df_not_correlated[df_not_correlated.index] == True].index
    df_out = df_in[un_corr_idx]
    return df_out

In [3]:
#Setting the Work Path
warnings.filterwarnings ('ignore')
#Work Path
os.chdir("C:\\Users\\A\\Desktop\\Bioactive\\EBV")

In [4]:
#AlvaDesc
from alvadesccliwrapper.alvadesc import AlvaDesc
#AlvaDesc Path(Linux default alvaDescCLI location)
aDesc = AlvaDesc('E:\\C++\\alvaDesc\\alvaDescCLI.exe') 

In [5]:
Pubchem_data=pd.read_csv("./Pubchem_data.csv",sep=',')
Iso_smiles=Pubchem_data['Iso_smiles']
print_RxC(Pubchem_data)
Pubchem_data.head()

11406 Moleculars × 3 Descriptors


,cid,Iso_smiles,Type
0,5273,C1CC(OC1)[N+]2=CNC3=C(N=CN=C32)N,Adenosine
1,13463,C1=NC(=C2C(=N1)N(C=N2)C3C(C(C(O3)CCl)O)O)N,Adenosine
2,39148,CN(C)C1[C@H](O[C@H](C1O)N2C=NC3=C(N=CN=C32)N)CO,Adenosine
3,56328,C1[C@H](O[C@H](C1F)N2C=NC3=C(N=CN=C32)N)CO,Adenosine
4,67358,C1=NC(=C2C(=N1)N(C=[N+]2[O-])[C@H]3[C@@H]([C@@...,Adenosine


In [6]:
Smis=[]
for i in Iso_smiles:
    Smis.append(i)
Mols = []
for Smi in Smis:
    Mol = Chem.MolFromSmiles(Smi)
    Mols.append(Mol)
Canonical_smis = []
for Mol in Mols:
    Canonical_smi = Chem.MolToSmiles(Mol)
    Canonical_smis.append(Canonical_smi)
Pubchem_data['Canonical_smiles']=Canonical_smis

In [7]:
#Load the data
Raw_data= pd.read_csv('./Original_data.csv',index_col=0)
X_scaled_data= pd.read_csv('./X_scaled_data.csv',index_col=0)
Transformer= joblib.load(filename='./Scaler_transformer.pkl')

In [8]:
Pubchem_data=Pubchem_data.drop_duplicates(subset="cid")
Pubchem_data=Pubchem_data.drop_duplicates(subset="Canonical_smiles")
Pubchem_data=Pubchem_data[~ Pubchem_data['Canonical_smiles'].isin(Raw_data['Canonical_smiles'])]
Pubchem_data.index=Pubchem_data['cid']
Pubchem_data.drop('cid',axis = 1,inplace = True)
print_RxC(Pubchem_data)
Pubchem_data.to_csv("./Pubchem_data_clearn.csv",sep=',',header=1,index=1)
Pubchem_data.head()

7240 Moleculars × 3 Descriptors


,Iso_smiles,Type,Canonical_smiles
cid,,,
5273,C1CC(OC1)[N+]2=CNC3=C(N=CN=C32)N,Adenosine,Nc1ncnc2c1[nH]c[n+]2C1CCCO1
13463,C1=NC(=C2C(=N1)N(C=N2)C3C(C(C(O3)CCl)O)O)N,Adenosine,Nc1ncnc2c1ncn2C1OC(CCl)C(O)C1O
39148,CN(C)C1[C@H](O[C@H](C1O)N2C=NC3=C(N=CN=C32)N)CO,Adenosine,CN(C)C1C(O)[C@H](n2cnc3c(N)ncnc32)O[C@@H]1CO
56328,C1[C@H](O[C@H](C1F)N2C=NC3=C(N=CN=C32)N)CO,Adenosine,Nc1ncnc2c1ncn2[C@@H]1O[C@H](CO)CC1F
67358,C1=NC(=C2C(=N1)N(C=[N+]2[O-])[C@H]3[C@@H]([C@@...,Adenosine,Nc1ncnc2c1[n+]([O-])cn2[C@@H]1O[C@H](CO)[C@@H]...


In [9]:
X_scaled_data

,Ui,Eta_betaP,SM6_B(p),piID,SM04_EA(bo),SM5_B(p),SM6_B(m),SM6_B(v),Uc,nBM,...,CATS2D_04_DD,Eig15_EA(dm),PHI,SM14_EA(ri),JGI10,MATS8i,SpMin6_Bh(v),SM15_AEA(ed),F09[N-N],NssO
OPTUM_LAB_ID,,,,,,,,,,,,,,,,,,,,,
3185712.0,0.660860,0.4750,0.666072,0.351500,0.631196,0.650365,0.227632,0.637375,0.712780,0.418605,...,0.25,0.84483,0.176391,0.498928,0.356238,0.344442,0.780514,0.522476,0.125,0.166667
3532333.0,0.557528,0.3500,0.478613,0.323871,0.430159,0.463049,0.181383,0.437487,0.613172,0.302326,...,0.00,0.84483,0.080920,0.365366,0.247234,0.418779,0.389447,0.378264,0.000,0.000000
16830460.0,0.601664,0.4500,0.560336,0.511522,0.557349,0.549239,0.208041,0.539961,0.694908,0.395349,...,0.00,0.84483,0.138379,0.274961,0.358655,0.512348,0.617672,0.317477,0.000,0.166667
3921421.0,0.713339,0.5750,0.632967,0.359267,0.616598,0.622451,0.181186,0.614309,0.791011,0.534884,...,0.00,0.84483,0.146061,0.671125,0.314003,0.548961,0.697860,0.756994,0.000,0.333333
1314610.0,0.622248,0.4375,0.513066,0.455372,0.545910,0.488266,0.174559,0.496682,0.676120,0.372093,...,0.00,0.84483,0.060975,0.363461,0.208924,0.499524,0.460734,0.371951,0.000,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664630.0,0.392361,0.2000,0.309936,0.219096,0.284890,0.297921,0.073210,0.289615,0.441011,0.162791,...,0.00,0.84483,0.064795,0.346192,0.599010,0.424263,0.489214,0.334631,0.000,0.000000
3228536.0,0.679022,0.4000,0.598130,0.376780,0.608750,0.567318,0.207894,0.565396,0.746103,0.465116,...,0.00,0.84483,0.089382,0.396462,0.292866,0.507947,0.614072,0.429004,0.000,0.166667
666815.0,0.533775,0.3000,0.477734,0.367866,0.510109,0.463521,0.155055,0.476061,0.589531,0.279070,...,0.00,0.84483,0.089822,0.476325,0.292739,0.450545,0.600852,0.499279,0.000,0.166667


In [10]:
Canonical_smis=Pubchem_data['Canonical_smiles']

In [11]:
#Calculate the descriotors
aDesc.set_input_SMILES(Canonical_smis.tolist())
if not aDesc.calculate_descriptors('ALL'):
 print('Error: ' + aDesc.get_error())
else:
 Data_all=aDesc.get_output()

In [12]:
Data_all=pd.DataFrame(Data_all)
Desc_names_names=aDesc.get_output_descriptors()
Data_all.columns=Desc_names_names
Data_all.index=Pubchem_data.index

In [13]:
X_test=Data_all[X_scaled_data.columns]
X_test.head()

,Ui,Eta_betaP,SM6_B(p),piID,SM04_EA(bo),SM5_B(p),SM6_B(m),SM6_B(v),Uc,nBM,...,CATS2D_04_DD,Eig15_EA(dm),PHI,SM14_EA(ri),JGI10,MATS8i,SpMin6_Bh(v),SM15_AEA(ed),F09[N-N],NssO
cid,,,,,,,,,,,,,,,,,,,,,
5273,2.584963,7.5,8.465290,8.287336,6.742143,7.205549,8.802058,8.542473,3.459432,10.0,...,0.0,0.0,2.072950,15.333924,0.0,0.191548,1.044409,28.909628,0.0,1.0
13463,2.584963,8.5,8.639362,8.761969,6.847342,7.391460,9.163389,8.711500,3.459432,10.0,...,0.0,0.0,3.220551,15.848795,0.0,-0.128820,1.107150,29.443620,0.0,1.0
39148,2.584963,8.5,8.678132,8.936284,6.882822,7.435133,9.035455,8.760417,3.459432,10.0,...,0.0,0.0,3.708532,16.098838,0.0,0.178698,1.308126,29.658230,0.0,1.0
56328,2.584963,8.5,8.575280,8.662170,6.812758,7.324406,8.937986,8.656549,3.459432,10.0,...,0.0,0.0,2.799601,15.559633,0.0,-0.073703,1.137905,29.168218,0.0,1.0
67358,2.584963,7.5,8.666884,8.863011,6.934275,7.406374,9.051158,8.755296,3.459432,10.0,...,1.0,0.0,3.232744,16.122003,0.0,-0.116060,1.186714,29.657170,0.0,1.0


In [14]:
X_test_scaled=Transformer.transform(X_test)
X_test_scaled =pd.DataFrame(X_test_scaled)
X_test_scaled.columns=X_test.columns
X_test_scaled.index=X_test.index
X_test_scaled.to_csv("./X_test_data.csv",sep=',',header=1,index=1)
print_RxC(X_test_scaled)
X_test_scaled.head()

7240 Moleculars × 1464 Descriptors


,Ui,Eta_betaP,SM6_B(p),piID,SM04_EA(bo),SM5_B(p),SM6_B(m),SM6_B(v),Uc,nBM,...,CATS2D_04_DD,Eig15_EA(dm),PHI,SM14_EA(ri),JGI10,MATS8i,SpMin6_Bh(v),SM15_AEA(ed),F09[N-N],NssO
cid,,,,,,,,,,,,,,,,,,,,,
5273,0.320978,0.1125,0.203392,0.236791,0.356817,0.164685,0.060085,0.198136,0.475923,0.186047,...,0.00,0.84483,0.005617,0.286898,0.0,0.613359,0.287299,0.386773,0.0,0.166667
13463,0.320978,0.1375,0.287919,0.282781,0.405009,0.257145,0.122657,0.281840,0.475923,0.186047,...,0.00,0.84483,0.033807,0.365041,0.0,0.260696,0.335336,0.458125,0.0,0.166667
39148,0.320978,0.1375,0.306746,0.299671,0.421263,0.278865,0.100503,0.306064,0.475923,0.186047,...,0.00,0.84483,0.045793,0.402991,0.0,0.599214,0.489214,0.486801,0.0,0.166667
56328,0.320978,0.1375,0.256802,0.273111,0.389166,0.223797,0.083624,0.254627,0.475923,0.186047,...,0.00,0.84483,0.023466,0.321154,0.0,0.321369,0.358884,0.421326,0.0,0.166667
67358,0.320978,0.1125,0.301284,0.292571,0.444835,0.264563,0.103222,0.303528,0.475923,0.186047,...,0.25,0.84483,0.034106,0.406507,0.0,0.274742,0.396255,0.486659,0.0,0.166667
